## **使用 Python Madmom 包，Adapter EIM 插件与 Scratch 流动生成频谱图（spectrogram）**

**1. EIM 快速版**

EIM 相比 Linda，可以更高频率地向 Scratch 发送数据，保证 Scratch 画图与 Python 数据分析的同步性，即实时性。

In [ ]:
import madmom
import numpy as np
import time
from codelab_adapter_client import AdapterNode

class MyNode(AdapterNode):
    NODE_ID = "eim/list_data"

    def __init__(self):
        super().__init__()
    
    def send_data(self, content):
        message = self.message_template()
        message["payload"]["content"] = content
        self.publish(message)

node = MyNode()
node.receive_loop_as_thread()
time.sleep(0.1)


kwargs = dict(
    sample_rate=44100,
    num_channels=1,
    frame_size=2048,
    hop_size=441,
    filterbank=madmom.audio.filters.MelFilterbank,
    num_bands=12
    
)
stream=signal.Stream(**kwargs)


spec_list = [i for i in range(10)]

n = 0

try:
    for frames in stream:  
        fs = madmom.audio.signal.FramedSignal(frames,**kwargs)
        stft = madmom.audio.stft.STFT(fs)
        spec = madmom.audio.spectrogram.LogarithmicFilteredSpectrogram(stft, **kwargs)
        i = n%2
        if i == 0:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            n = n + 1
        elif i == 1:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            #print(lindaout)
            node.send_data(spec_list)
            n = n + 1   
            
            

except KeyboardInterrupt:
    print('interrupt by user')


</br>

**2. Linda 滞后版**

In [ ]:
import madmom
import numpy as np
from madmom.audio import signal, hpss, spectrogram
import time
from codelab_adapter_client import AdapterNode



class MyNode(AdapterNode):
    NODE_ID = "linda/test"

    def __init__(self):
        super().__init__()
        
node = MyNode()
node.receive_loop_as_thread()


kwargs = dict(
    sample_rate=44100,
    num_channels=1,
    frame_size=2048,
    hop_size=441,
    filterbank=madmom.audio.filters.MelFilterbank,
    num_bands=12
    
)
stream=signal.Stream(**kwargs)


spec_list = [i for i in range(10)]

n = 0

try:
    for frames in stream:  
        lindaout = ["LindaOut",0]
        fs = madmom.audio.signal.FramedSignal(frames,**kwargs)
        stft = madmom.audio.stft.STFT(fs)
        spec = madmom.audio.spectrogram.LogarithmicFilteredSpectrogram(stft, **kwargs)
        i = n%2
        if i == 0:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            n = n + 1
        elif i == 1:
            for j in range(len(spec)):
                spec_list[i*5+j]=np.ndarray.tolist(spec[j])
            lindaout[1]=spec_list
            #print(lindaout)
            node.linda_out(lindaout)
            n = n + 1   
            
            

except KeyboardInterrupt:
    print('interrupt by user')
